# Text Search Engine
In this example we will be going over the code used to build a Text Search Engine. This example uses a modified BERT model to convert text to vectors stored in Milvus, which can then be combined with Milvus to search for similar text to the user input text.

### Sourced from Milvus Bootcamp
- https://github.com/milvus-io/bootcamp/tree/master/solutions/text_search_engine
- https://github.com/milvus-io/bootcamp/tree/master/solutions/question_answering_system

## Data

This example uses the English News dataset. In this example, we use a small subset of the dataset containing 180 mutually corresponding title-texts, which can be found in the **Data** directory.

## Requirements


|  Packages   |  Servers    |
|-                  | -                 |   
| pymilvus==2.0.0rc5      | milvus 2.0.0-rc5    |
| sentence_transformers      | postgres          |
| psycopg2          |
| pandas           |
| numpy   |

We have included a `requirements.txt` file in order to easily satisfy the required packages. 


## Up and Running

### Installing Packages
Install the required python packages with `requirements.txt`.

In [ ]:
pip install -r requirements.txt

### Starting Milvus Server

This demo uses Milvus 2.0, please refer to the [Install Milvus](https://milvus.io/cn/docs/install_standalone-docker.md) guide to learn how to use this docker container. For this example we wont be mapping any local volumes. 

In [ ]:
!wget https://raw.githubusercontent.com/milvus-io/milvus/master/deployments/docker/standalone/docker-compose.yml -O docker-compose.yml
!sudo docker-compose up -d

### Starting Postgres Server
For now, Milvus doesn't support storing multiple attributes for the data. Because of this we have to use another service to store these attributes and search through them, in this case PostgreSQL. 

In [1]:
! docker run --name postgres0 -d  -p 5438:5432 -e POSTGRES_HOST_AUTH_METHOD=trust postgres

db3d2bcec8f2bd24a04c4e5aa87b792c3cdd50f4957cdeb88290029fb77795fd


In [2]:
! docker logs postgres0 --tail 6

2022-10-16 02:02:31.819 UTC [1] LOG:  starting PostgreSQL 15.0 (Debian 15.0-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit
2022-10-16 02:02:31.819 UTC [1] LOG:  listening on IPv4 address "0.0.0.0", port 5432
2022-10-16 02:02:31.819 UTC [1] LOG:  listening on IPv6 address "::", port 5432
2022-10-16 02:02:31.820 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"
2022-10-16 02:02:31.823 UTC [62] LOG:  database system was shut down at 2022-10-16 02:02:31 UTC
2022-10-16 02:02:31.826 UTC [1] LOG:  database system is ready to accept connections


## Code Overview

### Connecting to Servers
We first start off by connecting to the servers. In this case the docker containers are running on localhost and the ports are the default ports. 

In [5]:
# Connecting to Milvus, BERT and Postgresql
from pymilvus import connections
import psycopg2
connections.connect(host='localhost', port='19530')
conn = psycopg2.connect(host='localhost', port='5438', user='postgres', password='postgres')
cursor = conn.cursor()

### Creating Collection and Setting Index
#### 1. Creating the Collection    
The next step is to create a collection, which requires declaring the name of the collection and the dimension of the vector.

In [11]:
from pymilvus import Collection, CollectionSchema, FieldSchema, DataType, utility

table_name= "alignment_lit"
field_name = "vector_embedding"

if utility.has_collection(table_name):
    utility.drop_collection(table_name)

pk = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)
field = FieldSchema(name=field_name, dtype=DataType.FLOAT_VECTOR, dim=768)
schema = CollectionSchema(fields=[pk,field], description="AI Alignment Literature Dataset")
collection = Collection(name=table_name, schema=schema)

#### 2. Setting an Index
After creating the collection we want to assign it an index type. This can be done before or after inserting the data. When done before, indexes will be made as data comes in and fills the data segments. In this example we are using IVF_SQ8 which requires the 'nlist' parameter.

In [12]:
index_param = {
        "metric_type":"L2",
        "index_type":"IVF_SQ8",
        "params":{"nlist":1024}
    }
collection.create_index(field_name=field_name, index_params=index_param)

Status(code=0, message='')

### Creating Table in Postgres  
PostgresSQL will be used to store Milvus ID and its corresponding title and text.

In [13]:
#Deleting previouslny stored table for clean run
drop_table = "DROP TABLE IF EXISTS " + table_name
cursor.execute(drop_table)
conn.commit()

try:
    sql = "CREATE TABLE if not exists " + table_name + " (ids bigint, title text, authors text, url text, text text);"
    cursor.execute(sql)
    conn.commit()
    print("create postgres table successfully!")
except Exception as e:
    print("can't create a postgres table: ", e)

create postgres table successfully!


### Processing and Storing the News Data
#### 1. Generating Embeddings
In this example we are using the sentence_transformer library  to encode the sentence into vectors. This library uses a modified BERT model to generate the embeddings, and in this example we are using a model pretrained using Microsoft's `mpnet`. More info can be found [here](https://www.sbert.net/docs/pretrained_models.html#sentence-embedding-models).

In [14]:
import json
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize

model = SentenceTransformer('allenai-specter')
# Get questions and answers.
data = pd.read_json('arxiv_pos_list.json')
title_data = data['title'].tolist()
text_data = data['abstract'].tolist()

title_text_data = data['title'].map(str) + '[SEP]' + data['abstract'].map(str)

sentence_embeddings = model.encode(title_text_data)
sentence_embeddings = normalize(sentence_embeddings)
print(type(sentence_embeddings))

<class 'numpy.ndarray'>


#### 2. Inserting Vectors into Milvus
Since this example dataset contains only 100 vectors, we are inserting all of them as one batch insert.

In [15]:
em =list(sentence_embeddings)
mr = collection.insert([em])
ids = mr.primary_keys
dicts ={}

In [16]:
len(ids)

959

#### 3. Inserting IDs and Title-text into PostgreSQL
In order to transfer the data into Postgres, we are creating a new file that combines all the data into a readable format. Once created, we pass this file into the Postgress server through STDIN due to the Postgres container not having access to the file locally. 

In [21]:
import os 
import re

# conn.rollback()
# extra delimiters | and quotes in title/text causes parsing issues, must strip
def clean_string(old_string):
    return old_string.replace("|","").replace("\"","").replace("'","")


def record_temp_csv(fname, ids, title, authors, urls, text):
    with open(fname,'w') as f:
        for i in range(len(ids)):
            line = str(ids[i]) + "|" + clean_string(title[i]) + "|" + ", ".join(authors[i]) + \
            "|" + clean_string(urls[i]) + "|" + clean_string(text[i]) + "\n"
            f.write(line)

def copy_data_to_pg(table_name, fname, conn, cur):
    fname = os.path.join(os.getcwd(),fname)
    try:
        sql = "COPY " + table_name + " FROM STDIN DELIMITER '|' CSV HEADER"
        cursor.copy_expert(sql, open(fname, "r"))
        conn.commit()
        print("Inserted into Postgress Sucessfully!")
    except Exception as e:
        print("Copy Data into Postgress failed: ", e)
        
DATA_WITH_IDS = 'arxiv_pos_list.csv'   

record_temp_csv(DATA_WITH_IDS, ids, title_data, data['authors'].tolist(), data['url'].tolist(), text_data)
copy_data_to_pg(table_name, DATA_WITH_IDS, conn, cursor)

Inserted into Postgress Sucessfully!


### Search
#### 1. Processing Query
When searching for a question, we first put the question through the same model to generate an embedding. Then with that embedding vector we  can search for similar embeddings in Milvus.  

In [25]:
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

title = "Could regulating the creators deliver trustworthy AI?"
# title = "Fooling the primate brain"

query_embeddings = []
embed = model.encode(title)
embed = embed.reshape(1,-1)
embed = normalize(embed)
query_embeddings = embed.tolist()

collection.load()
results = collection.search(query_embeddings, field_name, param=search_params, limit=15, expr=None)

#### 2. Getting the Similar Titles
There may not have titles that are similar to the given one. So we can set a threshold value, here we use 0.5, and when the most similar distance retrieved is less than this value, a hint that the system doesn't include the relevant question is returned. We then use the result ID's to pull out the similar titles from the Postgres server and print them with their corresponding similarity score.

In [45]:
similar_titles = []

for result in results[0]:
    sql = "select title, url from " + table_name + " where ids = " + str(result.id) + ";"
    print("\n", result.id, end='')
    cursor.execute(sql)
    rows = cursor.fetchall()
    if len(rows):
        title, url = rows[0]
        similar_titles.append((title, url, result.distance))
        print(f' ({result.distance:.3f}) {url} {title}', end='')
       


 436701982162682773 (0.233) http://arxiv.org/abs/2006.14750v1 Could regulating the creators deliver trustworthy AI?
 436701982162682720 (0.312) http://arxiv.org/abs/2107.06641v3 Trustworthy AI: A Computational Perspective
 436701982162682627 (0.321) http://arxiv.org/abs/2101.02032v5 Socially Responsible AI Algorithms: Issues, Purposes, and Challenges
 436701982162682712 (0.336) http://arxiv.org/abs/2002.06276v1 Trustworthy AI
 436701982162682777 (0.337) http://arxiv.org/abs/2110.01167v2 Trustworthy AI: From Principles to Practices
 436701982162682568 (0.343) http://arxiv.org/abs/2112.07773v1 Filling gaps in trustworthy development of AI
 436701982162682700 (0.352) http://arxiv.org/abs/1905.04994v2 Governance by Glass-Box: Implementing Transparent Moral Bounds for AI Behaviour
 436701982162682624 (0.369) http://arxiv.org/abs/2204.13828v1 Designing for Responsible Trust in AI Systems: A Communication Perspective
 436701982162682038 (0.371) http://arxiv.org/abs/2110.06674v1 Truthful AI: 

#### 3. Get the text
After getting a list of similar titles, choose the one that you feel is closest to yours. Then you can use that title to find the corresponding text in Postgres.

In [64]:
title, _, _ = similar_titles[0]
sql = "select text, authors, url from " + table_name + " where title = '" + title + "';"
cursor.execute(sql)
text, authors, url = cursor.fetchone()
print(f'[ {title} ]( {url} )\n\n{authors}\n\n{text}')

[ Could regulating the creators deliver trustworthy AI? ]( http://arxiv.org/abs/2006.14750v1 )

Labhaoise Ni Fhaolain, Andrew Hines

Is a new regulated profession, such as Artificial Intelligence (AI) Architect who is responsible and accountable for AI outputs necessary to ensure trustworthy AI? AI is becoming all pervasive and is often deployed in everyday technologies, devices and services without our knowledge. There is heightened awareness of AI in recent years which has brought with it fear. This fear is compounded by the inability to point to a trustworthy source of AI, however even the term trustworthy AI itself is troublesome. Some consider trustworthy AI to be that which complies with relevant laws, while others point to the requirement to comply with ethics and standards (whether in addition to or in isolation of the law). This immediately raises questions of whose ethics and which standards should be applied and whether these are sufficient to produce trustworthy AI in any e

In [8]:
# from pymilvus import utility
# utility.drop_collection("question_answer")
# utility.drop_collection("text_collection")
# utility.drop_collection("alignment_lit")

In [48]:
from pymilvus import utility, Collection

colls = utility.list_collections()
for coll_name in colls:
    print("\nname =", coll_name)
    coll = Collection(coll_name)
#     info = coll.get_collection_info()
#     print("INFO\n", info)
#     stats = coll.get_collection_stats()
#     print("STATS\n", stats)    
    print("description =", coll.description)
    print("schema =", coll.schema)
    print("num_entities =", coll.num_entities)
    print("primary_field =", coll.primary_field)    


name = alignment_lit
description = AI Alignment Literature Dataset
schema = {
  auto_id: True
  description: AI Alignment Literature Dataset
  fields: [{
    name: id
    description: 
    type: 5
    is_primary: True
    auto_id: True
  }, {
    name: vector_embedding
    description: 
    type: 101
    params: {'dim': 768}
  }]
}
num_entities = 959
primary_field = {
    name: id
    description: 
    type: 5
    is_primary: True
    auto_id: True
  }


In [90]:
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize
from pymilvus import Collection, connections
import psycopg2
import pandas as pd

def lit_search(query):
    model = SentenceTransformer('allenai-specter')
    
    query_embeddings = []
    embed = model.encode(query)
    embed = embed.reshape(1,-1)
    embed = normalize(embed)
    query_embeddings = embed.tolist()

    connections.connect(host='localhost', port='19530')
    conn = psycopg2.connect(host='localhost', port='5438', user='postgres', password='postgres')
    cursor = conn.cursor()

    table_name = "alignment_lit"
    field_name = "vector_embedding"
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

    collection = Collection(table_name)
    collection.load()
    results = collection.search(query_embeddings, field_name, param=search_params, limit=5, expr=None)

    similar_titles = []
    response = ""

    for result in results[0]:
        sql = "select title from " + table_name + " where ids = " + str(result.id) + ";"
        cursor.execute(sql)
        rows = cursor.fetchall()
        if len(rows):
            title = rows[0][0]
            similar_titles.append((result.distance, title))
            
    results = pd.DataFrame(similar_titles)
    results.columns = ["Score", "Title"]
    results["Score"] = results["Score"].round(decimals = 3)
    
    score, title = similar_titles[0]
    sql = "select text, authors, url from " + table_name + " where title = '" + title + "';"
    cursor.execute(sql)
    text, authors, url = cursor.fetchone()
    response += f'[{title}]({url})\n\n{authors}\n\n{text}'

    return results, response

In [91]:
# out1, out2 = lit_search("open ai gym")
# print(out1)
# print(out2)

   Score                                              Title
0  0.328                                         OpenAI Gym
1  0.388       A Berkeley View of Systems Challenges for AI
2  0.392                    The AI Index 2021 Annual Report
3  0.416  Weak AI is Likely to Never Become Strong AI, S...
4  0.416                 The 30-Year Cycle In The AI Debate
[OpenAI Gym](http://arxiv.org/abs/1606.01540v1)

Greg Brockman, Vicki Cheung, Ludwig Pettersson, Jonas Schneider, John Schulman, Jie Tang, Wojciech Zaremba

OpenAI Gym is a toolkit for reinforcement learning research. It includes a growing collection of benchmark problems that expose a common interface, and a website where people can share their results and compare the performance of algorithms. This whitepaper discusses the components of OpenAI Gym and the design decisions that went into the software.


In [93]:
import gradio as gr

with gr.Blocks() as demo:
    title = gr.Textbox(label="Question")
    submit_btn = gr.Button("Search")
    output = [gr.DataFrame(label="Results", headers=["Score", "Title"]), gr.Markdown(label="Output")]
    submit_btn.click(fn=lit_search, inputs=title, outputs=output)

demo.launch(inline=True)
# demo.launch(inline=True, share=True)

Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7fc1a2ae8be0>, 'http://127.0.0.1:7869/', None)

```
There are similar questions in the database, here are the closest matches: 
('Trustworthy AI: A Computational Perspective', 0.3309140205383301)
('Could regulating the creators deliver trustworthy AI?', 0.3363613784313202)
('Designing for Responsible Trust in AI Systems: A Communication Perspective', 0.36429765820503235)
('Socially Responsible AI Algorithms: Issues, Purposes, and Challenges', 0.3735373020172119)
('Trustworthy AI', 0.373695433139801)
('Trustworthy AI: From Principles to Practices', 0.37520506978034973)
('Know Your Model (KYM): Increasing Trust in AI and Machine Learning', 0.37871670722961426)
('The corruptive force of AI-generated advice', 0.38148611783981323)
```

In [ ]:
# generator = gr.Interface.load("huggingface/gpt2")
# translator = gr.Interface.load("huggingface/t5-small")

# gr.Series(generator, translator).launch()  # this demo generates text, then translates it to German, and outputs the final result.

In [24]:
# cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")
# for item in cursor.fetchall():
#      print(item)
cursor.execute("SELECT count(*) FROM alignment_lit")
for item in cursor.fetchone():
   print(item)

# conn.rollback()
# cursor.execute("SELECT * FROM alignment_lit LIMIT 5")

# import psycopg2

# conn = None
# try:
#     conn = psycopg2.connect(host='localhost', port='5438', user='postgres', password='postgres')
#     cursor = conn.cursor()

#     cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'")

#     for table in cursor.fetchall():
#         print(table)
# except (Exception, psycopg2.DatabaseError) as error:
#         print(error)
# finally:
#     if cursor is not None:
#         cursor.close()
#         print('Cursor connection closed.')
#     if conn is not None:
#         conn.close()
#         print('Database connection closed.')

958


In [ ]:
conn.rollback()